In [1]:
!pip install transformers --quiet

In [7]:
!id="1U6g7mfAuDL19deR6uUqI1T1taqpKH8rd"; conf=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$id -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p'); wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$conf&id=$id" -O isot.csv && rm -rf /tmp/cookies.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2022-08-25 13:36:58--  https://docs.google.com/uc?export=download&confirm=t&id=1U6g7mfAuDL19deR6uUqI1T1taqpKH8rd
Resolving docs.google.com (docs.google.com)... 142.250.203.206, 2a00:1450:401b:80d::200e
Connecting to docs.google.com (docs.google.com)|142.250.203.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-34-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/igh3u93c4cgt2cgt0eg42if0s01mv4md/1661427375000/05536367513713019519/*/1U6g7mfAuDL19deR6uUqI1T1taqpKH8rd?e=download&uuid=6f9c9276-6a84-496f-8bcd-7408eb581dd2 [following]
--2022-08-25 13:36:59--  https://doc-10-34-docs.googleuserconten

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import RepeatedStratifiedKFold

data = pd.read_csv("raw/isot.csv",sep=",")
print("labels\n",data["label"].value_counts())
print("shape  \n",data.shape)

bootstrap_size = 0

if bootstrap_size != 0:
    bootstrap_factor = bootstrap_size / data.shape[0]
    bootstrap = np.random.uniform(size=data.shape[0]) < bootstrap_factor
    data = data.iloc[bootstrap]

X, y = data["text"].values, data["label"].values

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

labels
 1    23481
0    21417
Name: label, dtype: int64
shape  
 (44898, 6)


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 632.15it/s]

shapes X (44898,) y (44898,)


In [2]:
data.columns

Index(['Unnamed: 0', 'title', 'text', 'subject', 'date', 'label'], dtype='object')

In [4]:
data["subject"].value_counts()

politicsNews       11272
worldnews          10145
News                9050
politics            6841
left-news           4459
Government News     1570
US_News              783
Middle-east          778
Name: subject, dtype: int64

In [3]:
import transformers
transformers.logging.set_verbosity_error()
from torch.utils.data import DataLoader 

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import TFDistilBertModel, DistilBertTokenizerFast

from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np

from scipy.sparse import csr_matrix
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device:", device)

if "disilbert_model" not in locals():
    disilbert_tokenizer =  AutoTokenizer.from_pretrained("distilbert-base-uncased")
    disilbert_model = AutoModel.from_pretrained("distilbert-base-uncased")
    handle = disilbert_model.to(device)


class BERTEmbeddings(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer =  disilbert_tokenizer
        self.model = disilbert_tokenizer
        self.max_length = 256
        self.model_name = disilbert_model

    def fit(self, X=None, y=None):
        pass
    
    def encode(self, txt):
        return self.tokenizer(txt, max_length=self.max_length, 
                              truncation=True, padding=True, return_tensors="pt")

    def transform(self, X=None):
        dataloader = DataLoader(X, batch_size=4, shuffle=False)
        allembeds = []
        for batch in tqdm(dataloader):
            batchenc = disilbert_tokenizer(batch, max_length=256, truncation=True, padding=True, return_tensors="pt")
            input_ids = batchenc['input_ids'].to(device)
            attention_mask = batchenc['attention_mask'].to(device)
            batchout = disilbert_model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
            embeds = [vec[0].cpu().detach().numpy() for vec in batchout[1][-1]]
            allembeds.extend(embeds)
        return csr_matrix(allembeds)


class BertHead(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.head = LogisticRegression(class_weight='auto', max_iter=10000)

    def fit(self, X=None, y=None):
        self.head.fit(X, y)


    def transform(self, X=None):
        pass
    
    def predict(self, X=None):    
        return self.head.predict(X)



TypeError: expected string or bytes-like object

In [3]:
bert = BERTEmbeddings()
X_dstil_numpy = bert.transform(X).toarray()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 11225/11225 [01:51<00:00, 100.49it/s]


In [4]:
import pickle
# SAVE
with open('isot_BERTEmbeddings.pickle', 'wb') as handle:
    pickle.dump(X_dstil_numpy, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.linear_model import LogisticRegression

arr = []

for fold,j in enumerate(foldids):
    train = foldids[fold][1]
    test = foldids[fold][2]
    xin, yin = X_dstil_numpy[train], np.array(y[train])
    cls = BertHead()
    
    cls.fit(xin, yin)
    y_pred = cls.predict(X_dstil_numpy[test])

    bac = balanced_accuracy_score(y[test], y_pred)
    arr.append(bac)

    print(" BAC=", bac)

print(10*"-")
print("AVG. BAC=",np.mean(arr),"+/-",np.std(arr))

 BAC= 0.9965345180344146
 BAC= 0.9972061583316935
 BAC= 0.9971430441304487
 BAC= 0.9970440011052812
 BAC= 0.9970963500699332
 BAC= 0.9967253737283818
 BAC= 0.9967762048372072
 BAC= 0.9965976054149235
 BAC= 0.9960055519997562
 BAC= 0.997696730273363
----------
AVG. BAC= 0.9968825537925403 +/- 0.00043757340219715397
